In [1]:
import numpy as np
import pandas as pd
import json
import os
from PIL import Image

import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline

In [2]:
def load_json(jfile):
    with open(jfile, 'rb') as f:
        return json.load(f)

In [3]:
path = '../retail_product_checkout/'

images_test = path+'test2019'
images_train = path+'train2019'
images_val = path+'val2019'

test = path+'instances_test2019.json'
train = path+'instances_train2019.json'
val = path+'instances_val2019.json'

test_data = load_json(test)
train_data = load_json(train)
val_data = load_json(val)

In [4]:
import os

paths = ['RPC',
         'RPC/train',
         'RPC/train/images',
         'RPC/train/labels',
         'RPC/test',
         'RPC/test/images',
         'RPC/test/labels',
         'RPC/valid',
         'RPC/valid/images',
         'RPC/valid/labels',
         'RPC/support',
         'RPC/support/images']

for path in paths:
    if not os.path.exists(path):
        os.mkdir(path)

In [5]:
from PIL import Image, ImageOps

def resize_with_padding(img, expected_size):
    img.thumbnail((expected_size[0], expected_size[1]))
    delta_width = expected_size[0] - img.size[0]
    delta_height = expected_size[1] - img.size[1]
    pad_width = delta_width // 2
    pad_height = delta_height // 2
    padding = (pad_width, pad_height, delta_width - pad_width, delta_height - pad_height)
    return ImageOps.expand(img, padding)


# PROCESSING LABEL

In [6]:
images_df = pd.DataFrame(train_data['images'])
anns_df = pd.DataFrame(train_data['annotations'])
category = pd.DataFrame(train_data['categories'])

all = pd.merge(anns_df, images_df, left_on='image_id', right_on='id')

def main():
    for i in tqdm(range(len(category))):
        id_list = (all.loc[anns_df['category_id'] == i+1])#prend tout les image de la catégorie X
        id_list = id_list[id_list['file_name'].str.contains("camera1-11.jpg")] # ( une face ,une back * 3 direction)
        if(len(id_list) <= 8):
            id_list = id_list.sort_values(by=['file_name'], ascending = True)
        else:        
            id_list =  id_list.sort_values(by=['file_name'], ascending = False)  

        for index, row in id_list.iterrows():
            xmin, ymin, width, height = row['bbox']
            image = Image.open(images_train+'/'+row['file_name'])
            #Crop for Label
            image = image.crop((int(xmin), int(ymin), int(xmin+width), int(ymin+height)))
            #Padding for (x,x)
            img = resize_with_padding(image, (640,640)) #try with 64x64
            #Save image
            img.save('RPC/support/images/'+str(i+1)+'.png')
main()

100%|██████████| 200/200 [00:15<00:00, 12.85it/s]


# PROCESSING IMAGE

In [7]:
category = pd.DataFrame(test_data['categories'])

images_df1 = pd.DataFrame(test_data['images'])
anns_df1 = pd.DataFrame(test_data['annotations'])

images_df2 = pd.DataFrame(val_data['images'])
anns_df2 = pd.DataFrame(val_data['annotations'])

images_df = pd.concat([images_df1, images_df2])
anns_df = pd.concat([anns_df1,anns_df2])

nb_category = len(category)

In [8]:
all = pd.merge(anns_df, images_df, left_on='image_id', right_on='id')
all = all.sort_values(by=['category_id'])
all.shape

(367935, 13)

In [9]:
# plt.figure(figsize=(8,35))
# result = pd.merge(all, category, how="outer", left_on='category_id', right_on='id')
# result['name'].value_counts().plot.barh()

In [10]:
# test_merge = pd.merge(all, category, how="outer", left_on='category_id', right_on='id')
# result['supercategory'].value_counts().sort_index().plot.barh()

In [11]:
#classes = np.arange(200)
#np.random.shuffle(classes)
# or export classes
classes = np.array([192,13,55,48,141,85,36,195,150,187,6,97,148,183,135,89,198,74
,42,197,95,62,10,43,188,189,30,59,171,76,193,28,100,58,143,91
,67,79,26,32,146,176,70,96,75,144,133,190,145,18,164,104,7,1
,165,138,181,178,33,158,114,105,21,186,3,174,159,41,69,117,110,185
,12,175,82,54,24,92,72,119,157,77,194,128,122,121,113,180,87,52
,103,20,152,71,4,136,125,151,80,172,170,156,63,154,45,0,160,35
,112,46,90,107,109,51,14,106,37,83,168,101,5,81,53,22,9,137
,163,177,124,31,179,19,116,134,27,57,126,78,118,15,120,167,142,196
,147,131,23,86,68,38,184,166,65,39,139,84,34,127,47,140,199,94
,66,108,173,44,191,182,130,64,40,11,73,123,149,129,161,155,99,61
,93,169,50,153,8,162,132,16,2,115,88,56,98,25,49,102,111,17
,60,29])
classes += 1 # aucune classes 0, la première est 1 ^^
print(classes.shape)

(200,)


In [12]:
print(classes[:160].shape)
print(classes[160:].shape)

(160,)
(40,)


In [13]:
TrainValid = all.loc[all['category_id'].isin(classes[:160])]
Test = all.loc[all['category_id'].isin(classes[160:])]
print(TrainValid.shape, Test.shape)

(295431, 13) (72504, 13)


In [14]:
print(all['category_id'].unique().shape)

(200,)


In [15]:
print(TrainValid['category_id'].unique().shape)
print(Test['category_id'].unique().shape)

(160,)
(40,)


# PROCESSING JSON

In [16]:
Train_image = TrainValid.groupby(['file_name'])['file_name'].nunique().sample(frac=0.7)
Val_image = TrainValid.groupby(['file_name'])['file_name'].nunique().drop(Train_image.index)
Test_image = Test.groupby(['file_name'])['file_name'].nunique()

print(Train_image.shape, Val_image.shape, Test_image.shape)

(20958,) (8982,) (21561,)


In [17]:
Train = TrainValid.loc[TrainValid['file_name'].isin(Train_image.index)]
Valid = TrainValid.loc[TrainValid['file_name'].isin(Val_image.index)]
print(Train.shape, Valid.shape, Test.shape)

(206709, 13) (88722, 13) (72504, 13)


In [18]:
print(Train['category_id'].unique().shape)
print(Valid['category_id'].unique().shape)
print(Test['category_id'].unique().shape)

(160,)
(160,)
(40,)


In [19]:
train_merge = pd.merge(Train, category, how="left", left_on='category_id', right_on='id')
train_merge.groupby('category_id', as_index=False).first()

,category_id,area,bbox,id_x,image_id,iscrowd,segmentation,point_xy,file_name,width,height,id_y,level,supercategory,id,name
0,1,215063.19,"[996.09, 139.75, 522.86, 411.32]",285450,25805,0,[[]],"[1257.52, 345.41]",20180912-14-04-50-321.jpg,1794,1794,25805,hard,puffed_food,1,1_puffed_food
1,2,160916.20,"[643.05, 292.81, 466.15, 345.2]",148180,16791,0,[[]],"[876.12, 465.41]",20180920-13-22-12-1190.jpg,1852,1852,16791,medium,puffed_food,2,2_puffed_food
2,4,299776.23,"[425.78, 1293.77, 579.49, 517.31]",138974,16083,0,[[]],"[715.52, 1552.42]",20180919-14-06-48-1097.jpg,1828,1828,16083,medium,puffed_food,4,4_puffed_food
3,5,298469.76,"[307.1, 448.95, 605.76, 492.72]",15551,3035,0,[[]],"[609.98, 695.31]",20180827-16-06-11-293.jpg,1834,1834,3035,easy,puffed_food,5,5_puffed_food
4,6,325987.44,"[599.0, 108.62, 624.12, 522.31]",302977,26778,0,[[]],"[911.06, 369.77]",20180926-13-09-18-1871.jpg,1833,1833,26778,hard,puffed_food,6,6_puffed_food
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,195,19855.89,"[987.6, 1114.33, 217.79, 91.17]",82810,11161,0,[[]],"[1096.5, 1159.91]",20181022-16-45-44-3109.jpg,1811,1811,11161,medium,stationery,195,195_stationery
156,196,328121.74,"[1147.42, 1063.5, 571.0, 574.65]",26344,4656,0,[[]],"[1432.92, 1350.83]",20180827-14-24-15-224.jpg,1830,1830,4656,easy,stationery,196,196_stationery
157,197,123177.92,"[1006.01, 675.02, 343.53, 358.57]",116811,14272,0,[[]],"[1177.78, 854.3]",20180904-16-39-44-65.jpg,1843,1843,14272,medium,stationery,197,197_stationery
158,198,41751.38,"[734.45, 938.96, 136.76, 305.29]",25446,4527,0,[[]],"[802.83, 1091.61]",20180824-14-33-05-25.jpg,1850,1850,4527,easy,stationery,198,198_stationery


In [20]:
train_merge = pd.merge(Train, category, how="left", left_on='category_id', right_on='id')
valid_merge = pd.merge(Valid, category, how="left", left_on='category_id', right_on='id')
test_merge = pd.merge(Test, category, how="left", left_on='category_id', right_on='id')

# df = pd.DataFrame({'Train/Valid': train_merge.groupby('category_id', as_index=False).first()['supercategory'].value_counts(),
# #                   'Valid':  valid_merge.groupby('category_id', as_index=False).first()['supercategory'].value_counts(),
#                    'Test': test_merge.groupby('category_id', as_index=False).first()['supercategory'].value_counts()})
#df.plot.barh(stacked=True,color={"Train": "darkred", "Valid": "tab:olive",'Test':"darkblue"})
# df.plot.barh(stacked=True,color={"Train/Valid": "darkred", 'Test':"darkblue"})

In [21]:
# df = pd.DataFrame({'Train': train_merge['name'].value_counts(),
#                    'Valid': valid_merge['name'].value_counts(),
#                    'Test':test_merge['name'].value_counts()})
# df.plot.barh(stacked=True,figsize=(8,35),color={"Train": "darkred", "Valid": "tab:olive",'Test':"darkblue"})

In [22]:
def saveImage(list_image, merge, path):
    for name in tqdm(list_image.keys()):
        if(os.path.exists('../retail_product_checkout/test2019/'+name)):
            image = Image.open('../retail_product_checkout/test2019/'+name)
        else:
            image = Image.open('../retail_product_checkout/val2019/'+name)
        for c in merge.loc[merge['file_name'] == name]['category_id'].unique():
            image = resize_with_padding(image, (640,640))
            image.save(path+name[:-4]+'.'+str(c)+'.png')

In [23]:
saveImage(Train_image,train_merge,'RPC/train/images/')

100%|██████████| 20958/20958 [1:43:23<00:00,  3.38it/s]  


In [24]:
saveImage(Val_image,valid_merge,'RPC/valid/images/')

100%|██████████| 8982/8982 [43:43<00:00,  3.42it/s]


In [25]:
saveImage(Test_image,test_merge,'RPC/test/images/')

100%|██████████| 21561/21561 [42:51<00:00,  8.38it/s] 


In [26]:
def savetxt(list_image, merge, path):
    for name in tqdm(list_image.keys()):
        for c in merge.loc[merge['file_name'] == name]['category_id'].unique():

            annotation = merge.loc[merge['file_name'] == name]
            annotation = annotation[annotation['category_id'] == c]
            bbox = annotation['bbox']

            with open(path+name[:-4]+'.'+str(c)+'.txt', 'a') as f:
                for (x,y,w,h),weight,heigh in zip(bbox,annotation['width'],annotation['height']):
                    s = '0'+' '+str(x/weight+(w/weight)/2)+' '+str(y/heigh+(h/heigh)/2)+' '+str(w/weight)+' '+str(h/heigh)+' \n'
                    f.write(s)

In [27]:
savetxt(Train_image,train_merge, 'RPC/train/labels/')

100%|██████████| 20958/20958 [14:08<00:00, 24.71it/s]


In [28]:
savetxt(Val_image,valid_merge, 'RPC/valid/labels/')

100%|██████████| 8982/8982 [02:49<00:00, 52.86it/s]


In [29]:
savetxt(Test_image,test_merge, 'RPC/test/labels/')

100%|██████████| 21561/21561 [02:31<00:00, 142.61it/s]


In [30]:
# train: ../train/images
# val: ../valid/images
# test: ../test/images
# support: ../support/images

# nc: 1
# names: ['coca']

with open('RPC/data.yaml', 'a') as f:
    f.write('train: ../train/images \n')
    f.write('val: ../valid/images \n')
    f.write('test: ../test/images \n')
    f.write('support: ../support/images \n')
    f.write(' \n')
    f.write('nc: 1 \n')
    f.write('names: [\'object\']')

In [31]:
#nano ~/.config/Ultralytics/settings.yaml --> change the path to RPC

In [ ]:
#TODO change valid by test ????